In [3]:
# 필요한 nltk 라이브러리 다운로드
import nltk
nltk.download('punkt')
nltk.download('webtext')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /data/ydkim/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package webtext to /data/ydkim/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.
[nltk_data] Downloading package wordnet to /data/ydkim/nltk_data...
[nltk_data] Downloading package stopwords to /data/ydkim/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /data/ydkim/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# 토근화

## 문장 토큰화

In [4]:
para = "Hello everyone. It's good to see you. Let's start our text mining class!"

from nltk.tokenize import sent_tokenize

# 주어진 텍스트를 문장 단위로 토큰화. 주로 . ! ? 등을 이용
print(sent_tokenize(para))

['Hello everyone.', "It's good to see you.", "Let's start our text mining class!"]


## 단어 토큰화

In [5]:
from nltk.tokenize import word_tokenize

# 주어진 text를 word 단위로 tokenize함
print(word_tokenize(para))

['Hello', 'everyone', '.', 'It', "'s", 'good', 'to', 'see', 'you', '.', 'Let', "'s", 'start', 'our', 'text', 'mining', 'class', '!']


위 결과를 보면 마침표나 느낌표가 별도의 단어로 분리되어 있으며, It's는 It과 's로 분리됨을 볼 수 있다.NLTK의 WordPunctTokenizer를 사용해보자.

In [6]:
from nltk.tokenize import WordPunctTokenizer

print(WordPunctTokenizer().tokenize(para))

['Hello', 'everyone', '.', 'It', "'", 's', 'good', 'to', 'see', 'you', '.', 'Let', "'", 's', 'start', 'our', 'text', 'mining', 'class', '!']


word_tokenize와 달리 WordPunctTokenizer는 It's를 It, ', s의 세 토큰으로 분리하는 것을 볼 수 있다. 따라서 사용자는 토크나이저의 특성을 파악하고 자신의 목적에 맞는 토크나이저를 선택할 필요가 있다.

## 노이즈와 불용어 제거

In [7]:
from nltk.corpus import stopwords # 일반적으로 분석대상이 아닌 단어들
from nltk.tokenize import RegexpTokenizer

english_stops = set(stopwords.words('english')) # 반복이 되지 않도록 set으로 변환

text1 = "Sorry, I couldn't go to movie yesterday."

tokenizer = RegexpTokenizer("[\w']+")
tokens = tokenizer.tokenize(text1.lower()) # word_tokenize로 토큰화

# stopwords를 제외한 단어들만으로 list를 생성
result = [word for word in tokens if word not in english_stops]

print(result)

['sorry', 'go', 'movie', 'yesterday']


위 결과를 보면, I, couldn't, to가 제거된 것을 볼 수 있다. 즉 이 세 단어는 불용어 사전에 포함돼 있다는 것을 의미한다.

# 정규화

정규화는 같은 의미를 가진 동일한 단어이면서 다른 형태로 쓰여진 단어들을 통힐해 표준 단어로 만드는 작업을 말한다.

## 어간 추출

어간 추출은 '어형이 변형된 단어로부터 접사 등을 제거하고 그 단어의 어간을 분리해 내는 작업'을 말한다.  
영어와 우리말은 원리와 구조가 다르므로 어간 추출도 달라질 수밖에 없다.  
영어의 경우에는 명사가 복수형으로 기술된 것을 단수형으로 바꾸는 작업도 어간 추출에 포함된다.  
영어에 대한 어간추출 알고리즘으로틑 포터 스테머, 랭카스터 스테머 등이 잘 알려져 있다.

다음은 NLTK의 포터 스테머를 사용해 어간 추출을 하는 예다.

In [10]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
print(stemmer.stem('cooking'), stemmer.stem('cookery'), stemmer.stem('cookbooks'))

cook cookeri cookbook


위와 같이 스테머 알고리즘은 단어가 변형되는 규칙을 이용해 원형을 찾으므로, 그 결과가 항상 사전에 있는 올바른 단어가 되지는 않는다.  
중요한 것은 포터 스테머를 쓰면 모든 단어가 같은 규칙에 따라 변환된다는 것이다. 즉 변환된 단어가 올바른 단어가 아니더라도, 동일한 형태로 변환됐으므로 분석의 의도를 충족시킬 수 있다.

다음은 토큰화와 결합해 어간 추출을 하는 예이다.

In [11]:
from nltk.tokenize import word_tokenize

para = "Hello everyone. It's good to see you. Let's start our text mining class!"
tokens = word_tokenize(para) # 토큰화 실행
print(tokens)
result = [stemmer.stem(token) for token in tokens] # 모든 토큰에 대해 스테밍 실행
print(result)

['Hello', 'everyone', '.', 'It', "'s", 'good', 'to', 'see', 'you', '.', 'Let', "'s", 'start', 'our', 'text', 'mining', 'class', '!']
['hello', 'everyon', '.', 'it', "'s", 'good', 'to', 'see', 'you', '.', 'let', "'s", 'start', 'our', 'text', 'mine', 'class', '!']


## 표제어 추출

표제어 추출은 단어의 기본형으로 변환한다는 뜻이다.어간과 단어의 기본형의 차이는 사전에 나오는 단어인지 아닌지의 차이로 구분할 수 있다.  
영어에 대한 표제어 추출기로 유명한 어휘 데이터베이스인 WordNet을 이용한 WordNet lemmatizer가 잘 알려져 있다.  
다음은 표제어를 추출하는 예를 보여준다.

In [14]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /data/ydkim/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /data/ydkim/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /data/ydkim/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /data/ydkim/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /data/ydkim/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /data/ydkim/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /

True

In [15]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize('cooking'))
print(lemmatizer.lemmatize('cooking', pos='v')) # 품사를 지정
print(lemmatizer.lemmatize('cookery'))
print(lemmatizer.lemmatize('cookbooks'))

cooking
cook
cookery
cookbook


# 품사 태깅

품사 태깅은 형태소에 대해 품사를 파악해 부착하는 작업을 말한다.

## NLTK를 활용한 품사 태깅

영어로 된 텍스트에 대해 품사 태깅을 하고자 한다면 지금까지와 마찬가지로 NLTK를 쓰는 것이 가장 편하다.

nltk.pos_tag()은 토큰화된 결과에 대해 품사를 태깅해 (단어, 품사)로 구성된 튜플의 리스트로 품사 태깅 결과를 반환해준다.

In [16]:
import nltk
from nltk.tokenize import word_tokenize

tokens = word_tokenize("Hello everyone. It's good to see you. Let's start our text mining class!")
print(nltk.pos_tag(tokens))

[('Hello', 'NNP'), ('everyone', 'NN'), ('.', '.'), ('It', 'PRP'), ("'s", 'VBZ'), ('good', 'JJ'), ('to', 'TO'), ('see', 'VB'), ('you', 'PRP'), ('.', '.'), ('Let', 'VB'), ("'s", 'POS'), ('start', 'VB'), ('our', 'PRP$'), ('text', 'NN'), ('mining', 'NN'), ('class', 'NN'), ('!', '.')]


### 원하는 품사의 단어들만 추출

자연어 분석을 할 때에는 상황에 따라 명사만 필요하거나, 특정 품사들만 사용할 때가 있다. 이 경우에는 다음과 같은 방법으로 원하는 품사들을 골라낼 수 있다.

In [18]:
my_tag_set = ['NN', 'VB', 'JJ']
my_words = [word for word, tag in nltk.pos_tag(tokens) if tag in my_tag_set]
print(my_words)

['everyone', 'good', 'see', 'Let', 'start', 'text', 'mining', 'class']


## 한글 형태소 분석과 품사 태깅

In [21]:
from konlpy.tag import Okt
okt = Okt()

print(okt.pos('프로그램 설치를 했습니다. 뭐 하나 쉬운 게 없네'))

[('프로그램', 'Noun'), ('설치', 'Noun'), ('를', 'Josa'), ('했습니다', 'Verb'), ('.', 'Punctuation'), ('뭐', 'Noun'), ('하나', 'Noun'), ('쉬운', 'Adjective'), ('게', 'Noun'), ('없네', 'Adjective')]
